In [1]:
import pandas as pd
import numpy as np
from pandas import Series
from zipfile import ZipFile
import os 
import requests
#import seaborn as sns
#import matplotlib as mpl
import matplotlib.pyplot as plt

#dependency --> pandas, xlrd, openpyxl

In [ ]:
#For inflation
import cpi
from datetime import date

#Run below first time to get CPI rate
#cpi.update()

Please note some of the URL is dynamic and require regular update.

In [8]:
%matplotlib inline
#if files are available then dont need to download.
downloadData = 0

In [4]:
#Download ZHVI data
url = "https://files.zillowstatic.com/research/public_csvs/zhvi/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1635114063"
#Latest data from zillow
r = requests.get(url,allow_redirects=True)
open('../Data/zillowData.csv', 'wb').write(r.content)

1833609

In [5]:
df = pd.read_csv('../Data/zillowData.csv')
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31
0,102001,0,United States,Country,NaN,127202.0,127546.0,127908.0,128645.0,129389.0,...,271623.0,274979.0,278635.0,282495.0,287202.0,292730.0,298613.0,304008.0,308632.0,312728.0
1,394913,1,"New York, NY",Msa,NY,224341.0,225681.0,226887.0,229261.0,231472.0,...,516687.0,521426.0,525792.0,530533.0,536044.0,543211.0,551142.0,558738.0,564641.0,569191.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,231193.0,231998.0,233231.0,235576.0,238089.0,...,741209.0,748698.0,756569.0,767615.0,783282.0,803090.0,823992.0,837437.0,846549.0,851308.0
3,394463,3,"Chicago, IL",Msa,IL,170104.0,170505.0,171024.0,172064.0,173166.0,...,258211.0,260534.0,262648.0,265690.0,268442.0,272263.0,276430.0,280547.0,283620.0,286280.0
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,130035.0,130139.0,130225.0,130437.0,130658.0,...,276637.0,280005.0,284015.0,289524.0,295921.0,303225.0,311010.0,318887.0,326057.0,332118.0


In [10]:
#remove regiontype
df.drop(['RegionType'], axis=1,  inplace=True)

In [6]:
#https://data.world/zillow-data/crosswalk-between-zillow-and-federally-defined-regions
#Crosswalk between Zillow and Federally Defined Regions
#url_xtalk = "https://data.world/zillow-data/crosswalk-between-zillow-and-federally-defined-regions/file/CountyCrossWalk_Zillow.csv"
url_xtalk = "http://files.zillowstatic.com/research/public/CountyCrossWalk_Zillow.csv"
rx = requests.get(url_xtalk,allow_redirects=True)
open('../Data/zillowXTalk.csv', 'wb').write(rx.content)

229072

In [11]:
df_Xtalk = pd.read_csv('../Data/zillowXTalk.csv',sep=',', encoding='gbk')
df_Xtalk.head()

,CountyName,StateName,StateFIPS,CountyFIPS,MetroName_Zillow,CBSAName,CountyRegionID_Zillow,MetroRegionID_Zillow,FIPS,CBSACode
0,Hudson,New Jersey,34,17,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",1106,394913.0,34017,35620.0
1,Morris,New Jersey,34,27,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",1241,394913.0,34027,35620.0
2,Nassau,New York,36,59,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",1252,394913.0,36059,35620.0
3,Orange,New York,36,71,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",1290,394913.0,36071,35620.0
4,Putnam,New York,36,79,"New York, NY","New York-Newark-Jersey City, NY-NJ-PA",1343,394913.0,36079,35620.0


In [12]:
#drop unnecessary columns
df_Xtalk.drop(["StateFIPS", "CountyFIPS", "FIPS", "CBSAName", 'CountyRegionID_Zillow', "StateName", "MetroName_Zillow"], axis=1, inplace=True)

In [13]:
#remove empty row and drop duplicates
df_Xtalk.dropna(axis=0, subset=['MetroRegionID_Zillow', 'CBSACode'], inplace=True)
df_Xtalk['CBSACode'] = df_Xtalk['CBSACode'].astype('int')
df_Xtalk.drop_duplicates(subset=['MetroRegionID_Zillow', 'CBSACode'], inplace=True)

In [14]:
#merge zillow data with cross talk data
zillowCBSA = df.merge(df_Xtalk, left_on='RegionID', right_on='MetroRegionID_Zillow')
#drop MetroRegionID_Zillow

In [15]:
#mapping of column name to occupation
mapFile = pd.read_excel('../Data/colID2Occupation.xlsx')
mapX = {i[0]:i[1] for i in mapFile.values}

In [17]:
#merge ACS and geoData
if(downloadData==1):
    from acsScripts.download_2010 import *
    from acsScripts.download_2011 import *
    from acsScripts.download_2012 import *
    from acsScripts.download_2013 import *
    from acsScripts.download_2014 import *
    from acsScripts.download_2015 import *
    from acsScripts.download_2016 import *
    from acsScripts.download_2017 import *
    from acsScripts.download_2018 import *
    from acsScripts.download_2019 import *
    
    download_2010()
    download_2011()
    download_2012()
    download_2013()
    download_2014()
    download_2015()
    download_2016()
    download_2017()
    download_2018()
    download_2019()

acsEarningTotal = []
years = list(range(2010,2020))
for year in years:
    d = pd.read_csv("{}/acsEarning.csv".format(year))
    #acsEarningTotal.append(d)
    year_monthCol  = list(zillowCBSA.filter(regex="{}-.*".format(year)))
    varNames2Keep = ['RegionID','SizeRank','RegionName','StateName','CountyName','CBSACode'] + year_monthCol
    zillowYear = zillowCBSA[varNames2Keep].copy()
    zillowYear.reset_index(drop=True,inplace=True)
    zillowYear['medianHousePrice'] = zillowYear[year_monthCol].apply(lambda s: np.median(s), axis=1)
    zillowYear.drop(year_monthCol, axis=1, inplace=True)
    houseEarningData = zillowYear.merge(d, on = 'CBSACode')
    incomeCol = list(houseEarningData.filter(regex='B24011.*'))
    for f in incomeCol:
        houseEarningData[f] = pd.to_numeric(houseEarningData[f], errors='coerce')
        houseEarningData[f] = houseEarningData['medianHousePrice']/houseEarningData[f]

    houseEarningData.rename(columns = mapX, inplace=True)
    houseEarningData.drop(["LOGRECNO", "NAME"], axis=1, inplace=True)
    folderAdd = "./{}".format(year)
    houseEarningData.to_csv(os.path.join(folderAdd,"houseEarningRatioData.csv"), index=False)

#acsEarning = acsData.merge(geoData, on = 'LOGRECNO')

In [ ]:
#download mortgate data: The URL will keep changing so please update.
mortgageURL = ("https://fred.stlouisfed.org/graph/fredgraph.xls?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=445&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=670&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=MORTGAGE30US&scale=left&cosd=2006-01-01&coed=2021-11-10&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=1&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Weekly%2C%20Ending%20Thursday&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2021-11-13&revision_date=2021-11-13&nd=1971-04-02")
r = requests.get(mortgageURL,allow_redirects=True)
open('../Data/mortgage30US.xls', 'wb').write(r.content)

In [3]:
#open mortgage data
mortgage_df = pd.read_excel("../Data/mortgage30US.xls", skiprows=10)
mortgage_df['observation_date'] = pd.to_datetime(mortgage_df['observation_date'])

In [4]:
N = 30
i = 0.025
alpha = 0.30
beta = 0.20

#compute Affordability level
def getALvalue(alpha, beta, N, i):
    al = alpha/(1-beta)*((1- (1+i)**(-N))/i)
    return al

#get affordability level
def getALRatio(beta, N, i):
    al = 1/(1-beta)*((1- (1+i)**(-N))/i)
    return al


In [5]:
years = list(range(2010,2020))
alData = []
for year in years:
    d_df = mortgage_df[mortgage_df['observation_date'].apply(lambda x: x.year==year)]
    interestRate = d_df['MORTGAGE30US'].mean()
    alRatio = getALRatio(beta, N, interestRate/100)
    alData.append([year, alRatio])

In [6]:
alData_df = pd.DataFrame(data=alData, columns=["year", "alRatio"])
alData_df.to_csv("alRatioData.csv", index=False)